In [1]:
DATA_DIR = '/lustre/groups/ml01/workspace/louis.kuemmerle/projects/A1/data2/' # "./../"
DATA_VERSION = 'oct22'
RESULTS_DIR = '/lustre/groups/ml01/workspace/louis.kuemmerle/projects/A1/results/'
SHAM = True

########################################################################################
sham_str = '_wSham' if SHAM else ''

In [3]:
import os 
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import scanpy as sc
import anndata as ann
from scipy.sparse import issparse

from scipy.sparse import csr_matrix
from matplotlib import rcParams
from matplotlib import colors

import datetime

#plt.rcParams['figure.dpi'] = 300

#sc.logging.print_versions()

In [4]:
region_to_color = {
    "Calvaria"  : "#e377c2",
    "Brain"     : "#1f77b4",
    "Femur"     : "#ff7f0e",
    "Humerus"   : "#279e68",
    "Meninges"  : "#d62728",
    "Pelvis"    : "#aa40fc",
    "Scapula"   : "#8c564b",
    "Vertebra"  : "#b5bd61",
}

condition_to_color = {
    'MCAo'      : "#1f77b4",
    'Naive'     : "#ff7f0e",
    'Sham'      : "#279e68",
}

# Check two gene lists from proteomics


In [9]:
df_degran = pd.read_csv(Path(DATA_DIR,"230302_mouse_proteomics_genes","neutrophil_degranulation_genes.csv"),index_col=0)
degran_genes = df_degran["0"].tolist()
df_immune = pd.read_csv(Path(DATA_DIR,"230302_mouse_proteomics_genes","neutrophil_mediated_immunity_genes.csv"),index_col=0)
immune_genes = df_immune["0"].tolist()

In [16]:
genes = np.unique(degran_genes + immune_genes).tolist()
genes = [g for g in genes if g in adata.var_names]

In [13]:
adata = sc.read(DATA_DIR+f'cellxgene_{DATA_VERSION}{sham_str}_umaps.h5ad')

adata.obs['region'] = adata.obs['region'].cat.rename_categories({'Skull': 'Calvaria'})
adata.obs['condition'] = adata.obs['condition'].cat.rename_categories({'MCAO': 'MCAo'})
adata.obs['region_condition'] = adata.obs["region"].astype(str) + "_" + adata.obs["condition"].astype(str)

In [30]:
bones = ['Femur', 'Humerus', 'Pelvis', 'Scapula', 'Calvaria', 'Vertebra']

col_names = []
cols = []

col_names.append(f"all_bones_all_conditions")
a = adata[(adata.obs["region"].isin(bones)), genes].copy()
cols.append(np.mean(a.X.toarray(),axis=0))

for c in ["Naive","Sham","MCAo"]:
    
    col_names.append(f"all_bones_{c}")
    a = adata[(adata.obs["condition"] == c) & (adata.obs["region"].isin(bones)), genes].copy()
    cols.append(np.mean(a.X.toarray(),axis=0))


for c in ["Naive","Sham","MCAo"]:
    
    for r in adata.obs['region'].cat.categories:
        col_names.append(f"{r}_{c}")
        a = adata[(adata.obs["condition"] == c) & (adata.obs["region"] == r), genes].copy()
        cols.append(np.mean(a.X.toarray(),axis=0))

/opt/python/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [33]:
df = pd.DataFrame(index=col_names,data=cols,columns=genes).T

In [34]:
df

,all_bones_all_conditions,all_bones_Naive,all_bones_Sham,all_bones_MCAo,Brain_Naive,Femur_Naive,Humerus_Naive,Meninges_Naive,Pelvis_Naive,Scapula_Naive,...,Calvaria_Sham,Vertebra_Sham,Brain_MCAo,Femur_MCAo,Humerus_MCAo,Meninges_MCAo,Pelvis_MCAo,Scapula_MCAo,Calvaria_MCAo,Vertebra_MCAo
Actr2,0.648258,0.525615,0.674149,0.699725,0.238800,0.493579,0.522228,0.262955,0.530872,0.533156,...,0.587854,0.806273,0.325236,0.671012,0.743704,0.459081,0.715890,0.707093,0.635043,0.723009
Adam10,0.265424,0.315638,0.274475,0.237020,0.106430,0.335639,0.356643,0.073220,0.324523,0.294972,...,0.190296,0.323517,0.120653,0.250616,0.265769,0.129990,0.235453,0.237838,0.182453,0.238325
Adam8,0.371983,0.275288,0.396154,0.411163,0.004626,0.231145,0.221105,0.026618,0.249695,0.238135,...,0.469915,0.431005,0.023067,0.367104,0.370653,0.129864,0.427283,0.397481,0.490011,0.431461
Adgre5,0.278486,0.234763,0.297871,0.293049,0.031530,0.219114,0.233040,0.076690,0.228329,0.224545,...,0.266886,0.338247,0.033571,0.272363,0.304971,0.196234,0.306046,0.309563,0.278173,0.295180
Adgrg3,0.141539,0.101695,0.152146,0.157443,0.000765,0.104448,0.102606,0.011955,0.110612,0.091723,...,0.096146,0.170678,0.004483,0.158303,0.190637,0.026708,0.160450,0.158201,0.115856,0.155016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vat1,0.030569,0.027712,0.030571,0.031992,0.046906,0.033209,0.031568,0.041119,0.028179,0.025320,...,0.013481,0.027985,0.062394,0.038305,0.040406,0.087272,0.031770,0.029116,0.017708,0.030135
Vcl,0.228400,0.176842,0.236324,0.251143,0.099045,0.176516,0.173423,0.061517,0.179550,0.179598,...,0.161470,0.288644,0.075981,0.242011,0.281302,0.100658,0.265162,0.256221,0.187751,0.267180
Vcp,0.473715,0.486604,0.499620,0.457627,0.257134,0.505744,0.494013,0.321661,0.500059,0.485452,...,0.349403,0.565586,0.292005,0.472220,0.491755,0.345211,0.451867,0.468443,0.357174,0.485991
Xrcc5,0.010173,0.010633,0.010113,0.009966,0.027135,0.012651,0.011399,0.037677,0.009594,0.009840,...,0.010149,0.009521,0.026974,0.009252,0.012536,0.021887,0.009933,0.009551,0.009387,0.009157


In [35]:
sc_df_degran = pd.DataFrame(index=degran_genes,data=np.nan,columns=df.columns)
sc_df_immune = pd.DataFrame(index=immune_genes,data=np.nan,columns=df.columns)

valid_degran_genes = [g for g in degran_genes if g in adata.var_names]
valid_immune_genes = [g for g in immune_genes if g in adata.var_names]

sc_df_degran.loc[valid_degran_genes] = df.loc[valid_degran_genes]
sc_df_immune.loc[valid_immune_genes] = df.loc[valid_immune_genes]

In [40]:
Path(RESULTS_DIR,"230302_mouse_proteomics_genes").mkdir(parents=True, exist_ok=True)

sc_df_degran.to_csv(Path(RESULTS_DIR,"230302_mouse_proteomics_genes","sc_mean_expr_neutrophil_degranulation_genes.csv"))
sc_df_immune.to_csv(Path(RESULTS_DIR,"230302_mouse_proteomics_genes","sc_mean_expr_neutrophil_mediated_immunity_genes.csv"))


# Create table with mean expressions of all genes

In [4]:
adata = sc.read(DATA_DIR+f'cellxgene_{DATA_VERSION}{sham_str}_umaps.h5ad')

adata.obs['region'] = adata.obs['region'].cat.rename_categories({'Skull': 'Calvaria'})
adata.obs['condition'] = adata.obs['condition'].cat.rename_categories({'MCAO': 'MCAo'})
adata.obs['region_condition'] = adata.obs["region"].astype(str) + "_" + adata.obs["condition"].astype(str)

In [6]:
bones = ['Femur', 'Humerus', 'Pelvis', 'Scapula', 'Calvaria', 'Vertebra']

col_names = []
cols = []

col_names.append(f"all_bones_all_conditions")
a = adata[(adata.obs["region"].isin(bones))].copy()
cols.append(np.mean(a.X.toarray(),axis=0))

for c in ["Naive","Sham","MCAo"]:
    
    col_names.append(f"all_bones_{c}")
    a = adata[(adata.obs["condition"] == c) & (adata.obs["region"].isin(bones))].copy()
    cols.append(np.mean(a.X.toarray(),axis=0))


for c in ["Naive","Sham","MCAo"]:
    
    for r in adata.obs['region'].cat.categories:
        col_names.append(f"{r}_{c}")
        a = adata[(adata.obs["condition"] == c) & (adata.obs["region"] == r)].copy()
        cols.append(np.mean(a.X.toarray(),axis=0))

/opt/python/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [8]:
df = pd.DataFrame(index=col_names,data=cols,columns=adata.var_names).T

In [9]:
df

,all_bones_all_conditions,all_bones_Naive,all_bones_Sham,all_bones_MCAo,Brain_Naive,Femur_Naive,Humerus_Naive,Meninges_Naive,Pelvis_Naive,Scapula_Naive,...,Calvaria_Sham,Vertebra_Sham,Brain_MCAo,Femur_MCAo,Humerus_MCAo,Meninges_MCAo,Pelvis_MCAo,Scapula_MCAo,Calvaria_MCAo,Vertebra_MCAo
index,,,,,,,,,,,,,,,,,,,,,
Xkr4,0.000000,0.000000,0.000000,0.000000,0.007861,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.006764,0.000000,0.000000,0.000065,0.000000,0.000000,0.000000,0.000000
Rp1,0.000045,0.000057,0.000000,0.000056,0.000000,0.000000,0.000000,0.000000,0.000110,0.000196,...,0.000000,0.000000,0.000000,0.000034,0.000046,0.000114,0.000063,0.000121,0.000043,0.000054
Sox17,0.000020,0.000053,0.000000,0.000011,0.032389,0.000000,0.000000,0.001189,0.000000,0.000000,...,0.000000,0.000000,0.009382,0.000000,0.000000,0.001927,0.000066,0.000000,0.000000,0.000000
Mrpl15,0.084816,0.078530,0.097048,0.083388,0.087849,0.094567,0.100141,0.095947,0.075928,0.058668,...,0.059664,0.118299,0.116873,0.091969,0.104737,0.129882,0.084894,0.076394,0.064099,0.071940
Lypla1,0.090201,0.063954,0.100403,0.099480,0.055945,0.062629,0.067964,0.045913,0.067965,0.055836,...,0.076305,0.115752,0.045835,0.098189,0.111324,0.068985,0.105488,0.095792,0.082985,0.099734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC168977.2,0.000083,0.000047,0.000058,0.000111,0.000000,0.000000,0.000080,0.000000,0.000000,0.000096,...,0.000244,0.000000,0.000000,0.000080,0.000136,0.000241,0.000064,0.000197,0.000112,0.000109
AC168977.1,0.000682,0.000699,0.000586,0.000708,0.001747,0.001020,0.001104,0.000842,0.000462,0.000364,...,0.000566,0.000923,0.000240,0.000550,0.000961,0.001069,0.001130,0.000384,0.000611,0.000554
AC149090.1,0.235047,0.221815,0.222165,0.246446,0.362131,0.206669,0.212423,0.141777,0.222716,0.231830,...,0.230897,0.269108,0.275155,0.228541,0.261478,0.207631,0.246666,0.276437,0.241779,0.237726


In [10]:
df.to_csv(Path(RESULTS_DIR,"230302_mouse_proteomics_genes","sc_mean_expr_all_genes.csv"))

In [11]:
Path(RESULTS_DIR,"230302_mouse_proteomics_genes","sc_mean_expr_all_genes.csv")

PosixPath('/lustre/groups/ml01/workspace/louis.kuemmerle/projects/A1/results/230302_mouse_proteomics_genes/sc_mean_expr_all_genes.csv')